In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [5]:
img = cv2.imread("./Imagenes/7.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

In [35]:
# noise removal
kernel = np.ones((3,3),np.uint8)/9
opening = cv2.morphologyEx(thresh,cv2.MORPH_BLACKHAT,kernel, iterations = 2)
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.2*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0
markers = cv2.watershed(img, markers)
img[markers == -1] = [0, 0, 255]
cv2.imshow("img", cv2.resize(img, (900,700)))
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
kernel = np.ones((3, 3), np.uint8)/9
closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, 
                            kernel, iterations = 2) 
  
bg = cv2.dilate(closing, kernel, iterations = 1) 
  
dist_transform = cv2.distanceTransform(closing, cv2.DIST_L2, 0) 
ret, fg = cv2.threshold(dist_transform, 0.0002
                        * dist_transform.max(), 255, 0) 
cv2.imshow("img", cv2.resize(fg, (900,700)))
cv2.waitKey(0)
cv2.destroyAllWindows()

Código que funciona re bien

In [62]:
img2 = cv2.imread("./Imagenes/51.jpg")
grises = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
_,th =  cv2.threshold(grises, 120, 255, cv2.THRESH_BINARY)

cnts,_ = cv2.findContours(th, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 
contornos = cv2.drawContours(img2, cnts, -1, (255,0,0),2)
cv2.imshow("img", cv2.resize(contornos, (900,700)))
cv2.waitKey(0)
cv2.destroyAllWindows()
len(cnts)


24253